# <b>■ 7장. 머신러닝 (p.284)</b>
    파이썬으로 머신러닝 데이터 분석하는 방법
    
    머신러닝 데이터 분석 순서
        1. csv file - DataFrame 생성
        2. 결측치를 제거하거나 치환
        3. 범주형 데이터 - 숫자형 데이터로 변환
        4. 정규화 진행 (min-max / z-score)
        5. 데이터를 훈련 데이터 / 테스트 데이터로 분리
        6. 훈련 데이터로 모델 생성 (sklearn)
        7. 테스트 데이터로 예측
        8. 모형의 예측능력을 평가

In [25]:
# 예제1. seaborn의 타이타닉 생존자를 예측하는 머신러닝 모델을 kNN 알고리즘으로 생성
# 1. csv file - DataFrame 생성
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic') # 컬럼이 모두 다 출력될 수 있도록 열의 개수에 한도를 늘린다.

pd.set_option('display.max_columns',20)
df.head(), df.shape

(   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
 0         0       3    male  22.0      1      0   7.2500        S  Third   
 1         1       1  female  38.0      1      0  71.2833        C  First   
 2         1       3  female  26.0      0      0   7.9250        S  Third   
 3         1       1  female  35.0      1      0  53.1000        S  First   
 4         0       3    male  35.0      0      0   8.0500        S  Third   
 
      who  adult_male deck  embark_town alive  alone  
 0    man        True  NaN  Southampton    no  False  
 1  woman       False    C    Cherbourg   yes  False  
 2  woman       False  NaN  Southampton   yes   True  
 3  woman       False    C  Southampton   yes  False  
 4    man        True  NaN  Southampton    no   True  ,
 (891, 15))

In [26]:
# 2. 결측치를 제거하거나 치환
# 2.1 타이타닉 데이터프레임의 자료형을 확인
df.info()
# 2.2 결측치 확인
df.isnull().sum()
# 2.3 deck의 결측치는 전체 891개 중 688개나 되므로 삭제 / embark, embark_town은 같은 데이터이므로 embark_town 삭제
rdf = df.drop(['embark_town','deck'],axis=1)
rdf.columns.values
# 2.4 age의 결측치가 177개 -> 해당 행을 삭제
rdf = rdf.dropna(subset=['age'],how='any',axis=0,)
len(rdf), rdf.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


(714,
 survived      0
 pclass        0
 sex           0
 age           0
 sibsp         0
 parch         0
 fare          0
 embarked      2
 class         0
 who           0
 adult_male    0
 alive         0
 alone         0
 dtype: int64)

In [68]:
# 3. 범주형 데이터 - 숫자형 데이터로 변환
# 3.1 머신러닝 학습에 필요한 컬럼을 선택(feature selection)
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
ndf.head()
# 위의 컬럼중에서 2개(sex, embarked)가 범주형
# 3.2 범주형 데이터를 숫자로 변환(onehot encoding)
gender = pd.get_dummies(ndf.sex)
ndf = pd.concat([ndf, gender],axis=1)
oh_embarked = pd.get_dummies(ndf.embarked, prefix='town')
# prefix='town'이라고 해야 컬럼안에 도시명으로 컬럼명이 생성
ndf = pd.concat([ndf, oh_embarked], axis=1)
# 3.3 범주형 컬럼을 drop한다.
ndf = ndf.drop(['embarked','sex'],axis=1,)
ndf

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
885,0,3,39.0,0,5,1,0,0,1,0
886,0,2,27.0,0,0,0,1,0,0,1
887,1,1,19.0,0,0,1,0,0,0,1
889,1,1,26.0,0,0,0,1,1,0,0


In [30]:
# 4. 정규화 진행 (min-max / z-score)
# 4.1 독립변수와 종속변수로 분리
X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S']]
y = ndf.survived
# 4.2 표준화
from sklearn import preprocessing as ppc
X = ppc.StandardScaler().fit(X).transform(X)
X

array([[ 0.91123237, -0.53037664,  0.52457013, ..., -0.47180795,
        -0.20203051,  0.53740921],
       [-1.47636364,  0.57183099,  0.52457013, ...,  2.11950647,
        -0.20203051, -1.86077941],
       [ 0.91123237, -0.25482473, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53740921],
       ...,
       [-1.47636364, -0.73704057, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53740921],
       [-1.47636364, -0.25482473, -0.55170307, ...,  2.11950647,
        -0.20203051, -1.86077941],
       [ 0.91123237,  0.15850313, -0.55170307, ..., -0.47180795,
         4.94974747, -1.86077941]])

In [67]:
# 5. 데이터를 훈련 데이터 / 테스트 데이터로 분리
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=4)
# 훈련 7, 테스트 3으로 나눈다.
X_train.shape , X_test.shape 

((499, 9), (215, 9))

In [63]:
### 6. 훈련 데이터로 모델 생성 (sklearn)
from sklearn.neighbors import KNeighborsClassifier as knc
knn = knc(n_neighbors=13)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=13, p=2,
                     weights='uniform')

In [64]:
# 7. 테스트 데이터로 예측
y_hat = knn.predict(X_test)
y_hat[0:10]

array([0, 1, 1, 0, 0, 1, 0, 0, 1, 1], dtype=int64)

In [65]:
# 8. 모형의 예측능력을 평가
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat)
knn_matrix

array([[109,  15],
       [ 23,  68]], dtype=int64)

In [66]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score( y_test, y_hat)
accuracy

0.8232558139534883

### ※ 문제219. (오늘의 마지막 문제) 나이의 결측치를 호칭의 나이의 평균값으로 치환하게 되면 정확도가 더 올라가는지 확인하시오

In [123]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')
n_df = pd.read_csv('train.csv')
df = pd.concat([df,n_df['Name']],axis=1)
appel = df['Name'].str.split(',').str[1].str.split('.')
df['title'] = appel.str.get(0)

age_m = pd.DataFrame(round(df.groupby('title')['age'].mean(),1))
df = pd.merge(df, age_m, on='title',how='left')
df.age_x.fillna(df.age_y, inplace=True)

rdf = df.drop(['embark_town','deck'],axis=1)
rdf.columns.values

ndf = rdf[['survived','pclass','sex','age_x','sibsp','parch','embarked']]

gender = pd.get_dummies(ndf.sex)
ndf = pd.concat([ndf, gender],axis=1)

oh_embarked = pd.get_dummies(ndf.embarked, prefix='town')
ndf = pd.concat([ndf, oh_embarked], axis=1)
ndf = ndf.drop(['embarked','sex'],axis=1,)

X = ndf[['pclass','age_x','sibsp','parch','female','male','town_C','town_Q','town_S']]
y = ndf.survived

from sklearn import preprocessing as ppc
X = ppc.StandardScaler().fit(X).transform(X)
X

from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=4)

from sklearn.neighbors import KNeighborsClassifier as knc
knn = knc(n_neighbors=13)
knn.fit(X_train, y_train)

y_hat = knn.predict(X_test)

from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat)
print(knn_matrix)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score( y_test, y_hat)
print(accuracy)

[[170   8]
 [ 28  62]]
0.8656716417910447


In [69]:
# 예제1. seaborn의 타이타닉 생존자를 예측하는 머신러닝 모델을 kNN 알고리즘으로 생성
# 1. csv file - DataFrame 생성
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic') # 컬럼이 모두 다 출력될 수 있도록 열의 개수에 한도를 늘린다.

pd.set_option('display.max_columns',20)
df.head(), df.shape

# 2. 결측치를 제거하거나 치환
# 2.1 타이타닉 데이터프레임의 자료형을 확인
df.info()
# 2.2 결측치 확인
df.isnull().sum()
# 2.3 deck의 결측치는 전체 891개 중 688개나 되므로 삭제 / embark, embark_town은 같은 데이터이므로 embark_town 삭제
rdf = df.drop(['embark_town','deck'],axis=1)
rdf.columns.values
# 2.4 age의 결측치가 177개 -> 해당 행을 삭제
rdf = rdf.dropna(subset=['age'],how='any',axis=0,)
len(rdf), rdf.isnull().sum()

# 3. 범주형 데이터 - 숫자형 데이터로 변환
# 3.1 머신러닝 학습에 필요한 컬럼을 선택(feature selection)
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
ndf.head()
# 위의 컬럼중에서 2개(sex, embarked)가 범주형
# 3.2 범주형 데이터를 숫자로 변환(onehot encoding)
gender = pd.get_dummies(ndf.sex)
ndf = pd.concat([ndf, gender],axis=1)
oh_embarked = pd.get_dummies(ndf.embarked, prefix='town')
# prefix='town'이라고 해야 컬럼안에 도시명으로 컬럼명이 생성
ndf = pd.concat([ndf, oh_embarked], axis=1)
# 3.3 범주형 컬럼을 drop한다.
ndf = ndf.drop(['embarked','sex'],axis=1,)
ndf

# 4. 정규화 진행 (min-max / z-score)
# 4.1 독립변수와 종속변수로 분리
X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S']]
y = ndf.survived
# 4.2 표준화
from sklearn import preprocessing as ppc
X = ppc.StandardScaler().fit(X).transform(X)
X

# 5. 데이터를 훈련 데이터 / 테스트 데이터로 분리
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=4)
# 훈련 7, 테스트 3으로 나눈다.
X_train.shape , X_test.shape 

### 6. 훈련 데이터로 모델 생성 (sklearn)
from sklearn.neighbors import KNeighborsClassifier as knc
knn = knc(n_neighbors=13)
knn.fit(X_train, y_train)

# 7. 테스트 데이터로 예측
y_hat = knn.predict(X_test)
y_hat[0:10]

# 8. 모형의 예측능력을 평가
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat)
knn_matrix

from sklearn.metrics import accuracy_score
accuracy = accuracy_score( y_test, y_hat)
accuracy

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


0.8232558139534883